In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn import metrics
import numpy as np
from collections import Counter
from sklearn.utils import shuffle
import os

# Defining a simple LSTM Model for Text Classification
class TextLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout):
        super(TextLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        output, (hidden, _) = self.lstm(embedded)
        hidden = self.dropout(hidden[-1])
        return self.fc(hidden)


In [ ]:

# Define a function to train the model
# Function to train the model
def train_model(model, iterator, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    for texts, labels in iterator:
        texts, labels = texts.to(device), labels.to(device)  # Move data to the device
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# Function to evaluate the model
def evaluate_model(model, iterator, criterion, device):
    model.eval()
    epoch_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for texts, labels in iterator:
            texts, labels = texts.to(device), labels.to(device)  # Move data to the device
            predictions = model(texts)
            loss = criterion(predictions, labels)
            epoch_loss += loss.item()
            all_preds.append(predictions)
            all_labels.append(labels)
    avg_loss = epoch_loss / len(iterator)
    eval_metrics = eval_mod(torch.cat(all_preds), torch.cat(all_labels))
    return avg_loss, eval_metrics

def eval_mod(preds, labels):
    y_true = labels
    y_pred_label = torch.argmax(preds, dim=1)
    y_pred_label = y_pred_label.cpu().numpy()  # Ensuring it's a numpy array
    y_true = y_true.cpu().numpy()  # Ensuring labels are also numpy array

    accuracy = metrics.accuracy_score(y_true, y_pred_label)
    f1_weighted = metrics.f1_score(y_true, y_pred_label, average='weighted')
    f1_macro = metrics.f1_score(y_true, y_pred_label, average='macro')
    f1_micro = metrics.f1_score(y_true, y_pred_label, average='micro')
    precision_weighted = metrics.precision_score(y_true, y_pred_label, average='weighted')
    precision_macro = metrics.precision_score(y_true, y_pred_label, average='macro')
    precision_micro = metrics.precision_score(y_true, y_pred_label, average='micro')
    recall_weighted = metrics.recall_score(y_true, y_pred_label, average='weighted')
    recall_macro = metrics.recall_score(y_true, y_pred_label, average='macro')
    recall_micro = metrics.recall_score(y_true, y_pred_label, average='micro')

    results = {"accuracy": accuracy,
               "f1_weighted": f1_weighted,
               "f1_macro": f1_macro,
               "f1_micro": f1_micro,
               "precision_weighted": precision_weighted,
               "precision_macro": precision_macro,
               "precision_micro": precision_micro,
               "recall_weighted": recall_weighted,
               "recall_macro": recall_macro,
               "recall_micro": recall_micro
               }

    return results

In [ ]:
import numpy as np
from sklearn.utils import shuffle

def read_data(dataset_name, split_ratios=(0.8, 0.1, 0.1), seed=123):
    data = {}
    x, y = [], []

    # Define dataset files based on the dataset name
    if dataset_name == "r8_presplit":
        sentences_file = "drive/MyDrive/CPSC_577_FP/r8_sentences_clean.txt"
        labels_file = "drive/MyDrive/CPSC_577_FP/r8_labels.txt"
        label_pos = 2
    elif dataset_name == "ag_presplit":
        sentences_file = "drive/MyDrive/CPSC_577_FP/ag_sentences_clean.txt"
        labels_file = "drive/MyDrive/CPSC_577_FP/ag_labels.txt"
        label_pos = 2
    elif dataset_name == "twitter_asian_prejudice":
        sentences_file = "drive/MyDrive/CPSC_577_FP/twitter_asian_prejudice_sentences_clean.txt"
        labels_file = "drive/MyDrive/CPSC_577_FP/twitter_asian_prejudice_labels.txt"
        label_pos = 0
    else:
        raise ValueError("Invalid dataset name provided.")

    # Read sentence data
    with open(sentences_file, "r", encoding="utf-8") as f:
        for line in f:
            if line[-1] == "\n":
                line = line[:-1]
            x.append(line.split())

    # Read label data
    with open(labels_file, "r", encoding="utf-8") as d:
        for line in d:
            if line[-1] == "\n":
                line = line[:-1]
            y.append(line.split()[label_pos])

    # Shuffle data using the provided seed
    x, y = shuffle(x, y, random_state=seed)

    # Calculate split indices based on split ratios
    total_count = len(x)
    train_end = int(split_ratios[0] * total_count)
    valid_end = train_end + int(split_ratios[1] * total_count)

    # Split data into training, validation, and test sets
    data["train_x"], data["train_y"] = x[:train_end], y[:train_end]
    data["valid_x"], data["valid_y"] = x[train_end:valid_end], y[train_end:valid_end]
    data["test_x"], data["test_y"] = x[valid_end:], y[valid_end:]

    return data


In [ ]:

def create_data_loader(texts, labels, batch_size, vocab, label_dict, max_len=100):
    # Convert texts to integer sequences
    processed_texts = [[vocab[word] if word in vocab else vocab["<UNK>"] for word in text] for text in texts]

    # Pad or truncate sequences
    processed_texts = [text[:max_len] + [vocab["<PAD>"]] * (max_len - len(text)) if len(text) < max_len else text[:max_len] for text in processed_texts]

    # Convert labels to integers
    processed_labels = [label_dict[label] for label in labels]

    # Create tensors
    texts_tensor = torch.tensor(processed_texts, dtype=torch.long)
    labels_tensor = torch.tensor(processed_labels, dtype=torch.long)

    # Create dataset
    dataset = TensorDataset(texts_tensor, labels_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import warnings

def main():
    warnings.filterwarnings('ignore')
    dataset_name = "r8_presplit"
    # Parameters

    embedding_dim = 64
    hidden_dim = 50
    output_dim = 8  # Adjusted to the dataset's specific output dimensions
    num_layers = 2
    dropout = 0
    batch_size = 64
    num_epochs = 100
    patience = 10  # Early stopping patience
    num_experiments = 5
    random_seeds = [33, 15, 86, 109, 78]
    all_experiment_results = []

    for exp in range(num_experiments):
        random_seed = random_seeds[exp]
        # Load data

        data = read_data(dataset_name, seed = random_seed)
        vocab = {word: i for i, word in enumerate(set(sum(data['train_x'], [])))}  # Creating a simple vocab
        vocab["<PAD>"] = 0
        vocab["<UNK>"] = 1
        label_dict = {label: i for i, label in enumerate(set(data['train_y']))}
        vocab_size = len(vocab)

        # Create data loaders
        train_loader = create_data_loader(data['train_x'], data['train_y'], batch_size, vocab, label_dict)
        valid_loader = create_data_loader(data['valid_x'], data['valid_y'], batch_size, vocab, label_dict)
        test_loader = create_data_loader(data['test_x'], data['test_y'], batch_size, vocab, label_dict)


        # Create model
        model = TextLSTM(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)
        learning_rate = 0.1
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        criterion = nn.CrossEntropyLoss()

        # Early stopping mechanism
        best_valid_loss = float('inf')
        best_model_path = ""
        epochs_no_improve = 0

        # Training and validation
        for epoch in range(num_epochs):
            start_time = time.time()
            train_loss = train_model(model, train_loader, optimizer, criterion, device)
            train_time = time.time() - start_time
            valid_loss, valid_results = evaluate_model(model, valid_loader, criterion, device)

            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                epochs_no_improve = 0
                best_model_path = f'/content/drive/My Drive/CPSC_577_FP/logs/LSTM/model_best_{exp}.pt'
                torch.save(model.state_dict(), best_model_path)
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print("Early stopping triggered on experiment", exp+1)
                    break
        # Load the best model and evaluate on the test set
        model.load_state_dict(torch.load(best_model_path))
        test_loss, test_results = evaluate_model(model, test_loader, criterion, device)
        all_experiment_results.append(test_results)

    # Calculate mean and standard deviation across experiments
    final_metrics = {key: [] for key in all_experiment_results[0]}
    for results in all_experiment_results:
        for key in results:
            final_metrics[key].append(results[key])

    for metric in final_metrics:
        values = np.array(final_metrics[metric])
        mean = values.mean()
        std = values.std()
        print(f'{metric}: Mean={mean:.4f}, Std={std:.4f}')

In [ ]:
main()

Early stopping triggered on experiment 1
Early stopping triggered on experiment 2
Early stopping triggered on experiment 3
Early stopping triggered on experiment 4
Early stopping triggered on experiment 5
accuracy: Mean=0.8482, Std=0.0402
f1_weighted: Mean=0.8321, Std=0.0495
f1_macro: Mean=0.5028, Std=0.1018
f1_micro: Mean=0.8482, Std=0.0402
precision_weighted: Mean=0.8255, Std=0.0535
precision_macro: Mean=0.5127, Std=0.1101
precision_micro: Mean=0.8482, Std=0.0402
recall_weighted: Mean=0.8482, Std=0.0402
recall_macro: Mean=0.5179, Std=0.0940
recall_micro: Mean=0.8482, Std=0.0402


In [ ]:
def main1():
    warnings.filterwarnings('ignore')
    # Load data
    dataset_name = "twitter_asian_prejudice"


    # Parameters

    embedding_dim = 64
    hidden_dim = 50
    output_dim = 5  # Adjusted to the dataset's specific output dimensions
    num_layers = 2
    dropout = 0
    batch_size = 64
    num_epochs = 50
    patience = 10  # Early stopping patience
    num_experiments = 5
    random_seeds = [33, 15, 86, 109, 78]
    all_experiment_results = []

    for exp in range(num_experiments):
        random_seed = random_seeds[exp]
        data = read_data(dataset_name, seed= random_seed)
        vocab = {word: i for i, word in enumerate(set(sum(data['train_x'], [])))}  # Creating a simple vocab
        vocab["<PAD>"] = 0
        vocab["<UNK>"] = 1
        label_dict = {label: i for i, label in enumerate(set(data['train_y']))}
        vocab_size = len(vocab)
        # Create data loaders
        train_loader = create_data_loader(data['train_x'], data['train_y'], batch_size, vocab, label_dict)
        valid_loader = create_data_loader(data['valid_x'], data['valid_y'], batch_size, vocab, label_dict)
        test_loader = create_data_loader(data['test_x'], data['test_y'], batch_size, vocab, label_dict)


        # Create model
        model = TextLSTM(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)
        learning_rate = 0.1
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        criterion = nn.CrossEntropyLoss()

        # Early stopping mechanism
        best_valid_loss = float('inf')
        best_model_path = ""
        epochs_no_improve = 0

        # Training and validation
        for epoch in range(num_epochs):
            start_time = time.time()
            train_loss = train_model(model, train_loader, optimizer, criterion, device)
            train_time = time.time() - start_time
            valid_loss, valid_results = evaluate_model(model, valid_loader, criterion, device)

            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                epochs_no_improve = 0
                best_model_path = f'/content/drive/My Drive/CPSC_577_FP/logs/LSTM/model_best_{exp}.pt'
                torch.save(model.state_dict(), best_model_path)
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print("Early stopping triggered on experiment", exp+1)
                    break
        # Load the best model and evaluate on the test set
        model.load_state_dict(torch.load(best_model_path))
        test_loss, test_results = evaluate_model(model, test_loader, criterion, device)
        all_experiment_results.append(test_results)

    # Calculate mean and standard deviation across experiments
    final_metrics = {key: [] for key in all_experiment_results[0]}
    for results in all_experiment_results:
        for key in results:
            final_metrics[key].append(results[key])

    for metric in final_metrics:
        values = np.array(final_metrics[metric])
        mean = values.mean()
        std = values.std()
        print(f'{metric}: Mean={mean:.4f}, Std={std:.4f}')

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
main1()  # Run your main function again

Early stopping triggered on experiment 1
Early stopping triggered on experiment 2
Early stopping triggered on experiment 3
Early stopping triggered on experiment 4
Early stopping triggered on experiment 5
accuracy: Mean=0.6813, Std=0.0078
f1_weighted: Mean=0.5522, Std=0.0100
f1_macro: Mean=0.1621, Std=0.0011
f1_micro: Mean=0.6813, Std=0.0078
precision_weighted: Mean=0.4642, Std=0.0105
precision_macro: Mean=0.1363, Std=0.0016
precision_micro: Mean=0.6813, Std=0.0078
recall_weighted: Mean=0.6813, Std=0.0078
recall_macro: Mean=0.2000, Std=0.0000
recall_micro: Mean=0.6813, Std=0.0078
